# Rock-Paper-Scissors CNN Project
## 1. Data Exploration and Analysis

This notebook focuses on exploring the Rock-Paper-Scissors dataset, understanding its structure, and performing initial data analysis.


In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)


### Dataset Information

The Rock-Paper-Scissors dataset contains images of hand gestures for the three classes:
- **Rock**: Closed fist
- **Paper**: Open palm  
- **Scissors**: Two fingers extended (index and middle finger)

Let's explore the dataset structure and characteristics.


In [ ]:
# Define paths
data_path = Path('../data/raw')
classes = ['rock', 'paper', 'scissors']

# Check if data directory exists
if not data_path.exists():
    print(f"Data directory {data_path} does not exist.")
    print("Please download the dataset from Kaggle and place it in the data/raw directory.")
    print("Dataset URL: https://www.kaggle.com/datasets/drgfreeman/rockpaperscissors")
else:
    print(f"Data directory found: {data_path}")
    
    # Explore directory structure
    for class_name in classes:
        class_path = data_path / class_name
        if class_path.exists():
            num_images = len(list(class_path.glob('*.png')))
            print(f"{class_name}: {num_images} images")
        else:
            print(f"{class_name}: directory not found")


### Dataset Statistics

Let's analyze the distribution of images across classes and examine some sample images.


In [ ]:
# Collect dataset statistics
dataset_stats = {}
total_images = 0

for class_name in classes:
    class_path = data_path / class_name
    if class_path.exists():
        images = list(class_path.glob('*.png'))
        num_images = len(images)
        dataset_stats[class_name] = num_images
        total_images += num_images
        
        # Get image dimensions from first image
        if images:
            sample_img = Image.open(images[0])
            print(f"{class_name}: {num_images} images, sample size: {sample_img.size}")

print(f"\nTotal images: {total_images}")

# Create visualization of class distribution
if dataset_stats:
    plt.figure(figsize=(12, 5))
    
    # Bar plot
    plt.subplot(1, 2, 1)
    bars = plt.bar(dataset_stats.keys(), dataset_stats.values(), 
                   color=['#ff9999', '#66b3ff', '#99ff99'])
    plt.title('Distribution of Images by Class', fontsize=14, fontweight='bold')
    plt.xlabel('Class')
    plt.ylabel('Number of Images')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 10,
                f'{int(height)}', ha='center', va='bottom', fontweight='bold')
    
    # Pie chart
    plt.subplot(1, 2, 2)
    plt.pie(dataset_stats.values(), labels=dataset_stats.keys(), autopct='%1.1f%%', 
            colors=['#ff9999', '#66b3ff', '#99ff99'], startangle=90)
    plt.title('Class Distribution (Percentage)', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("\n" + "="*50)
    print("DATASET STATISTICS SUMMARY")
    print("="*50)
    for class_name, count in dataset_stats.items():
        percentage = (count / total_images) * 100
        print(f"{class_name.upper()}: {count:,} images ({percentage:.1f}%)")
    print(f"\nTOTAL: {total_images:,} images")
    print("="*50)


### Sample Images Visualization

Let's examine sample images from each class to understand the visual characteristics and variations.


In [ ]:
# Display sample images from each class
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
fig.suptitle('Sample Images from Each Class', fontsize=16, fontweight='bold')

for i, class_name in enumerate(classes):
    class_path = data_path / class_name
    if class_path.exists():
        images = list(class_path.glob('*.png'))
        
        for j in range(5):
            if j < len(images):
                img = Image.open(images[j])
                axes[i, j].imshow(img)
                axes[i, j].set_title(f'{class_name.capitalize()} {j+1}', fontweight='bold')
                axes[i, j].axis('off')
            else:
                axes[i, j].text(0.5, 0.5, 'No data', ha='center', va='center', 
                               transform=axes[i, j].transAxes, fontsize=12)
                axes[i, j].axis('off')
    else:
        for j in range(5):
            axes[i, j].text(0.5, 0.5, 'No data', ha='center', va='center',
                           transform=axes[i, j].transAxes, fontsize=12)
            axes[i, j].axis('off')

plt.tight_layout()
plt.show()


### Image Analysis

Let's analyze the technical characteristics of the images (dimensions, color channels, etc.).


In [ ]:
# Analyze image characteristics
image_analysis = {}

for class_name in classes:
    class_path = data_path / class_name
    if class_path.exists():
        images = list(class_path.glob('*.png'))
        
        if images:
            # Sample a few images for analysis
            sample_size = min(20, len(images))
            sample_images = np.random.choice(images, sample_size, replace=False)
            
            dimensions = []
            channels = []
            file_sizes = []
            
            for img_path in sample_images:
                img = Image.open(img_path)
                dimensions.append(img.size)  # (width, height)
                channels.append(len(img.getbands()))
                file_sizes.append(img_path.stat().st_size / 1024)  # KB
            
            image_analysis[class_name] = {
                'dimensions': dimensions,
                'channels': channels,
                'file_sizes': file_sizes,
                'sample_size': sample_size
            }

# Display analysis results
print("IMAGE CHARACTERISTICS ANALYSIS")
print("="*60)

for class_name, analysis in image_analysis.items():
    print(f"\n{class_name.upper()}:")
    print(f"  Sample size: {analysis['sample_size']}")
    print(f"  Unique dimensions: {set(analysis['dimensions'])}")
    print(f"  Color channels: {set(analysis['channels'])}")
    
    # Calculate statistics
    avg_width = np.mean([dim[0] for dim in analysis['dimensions']])
    avg_height = np.mean([dim[1] for dim in analysis['dimensions']])
    avg_file_size = np.mean(analysis['file_sizes'])
    
    print(f"  Average dimensions: {avg_width:.0f} x {avg_height:.0f}")
    print(f"  Average file size: {avg_file_size:.1f} KB")
    print(f"  File size range: {min(analysis['file_sizes']):.1f} - {max(analysis['file_sizes']):.1f} KB")

# Create visualization of image dimensions
if image_analysis:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, (class_name, analysis) in enumerate(image_analysis.items()):
        widths = [dim[0] for dim in analysis['dimensions']]
        heights = [dim[1] for dim in analysis['dimensions']]
        
        axes[i].scatter(widths, heights, alpha=0.7, s=50)
        axes[i].set_title(f'{class_name.capitalize()} - Image Dimensions')
        axes[i].set_xlabel('Width (pixels)')
        axes[i].set_ylabel('Height (pixels)')
        axes[i].grid(True, alpha=0.3)
        
        # Add average point
        avg_w, avg_h = np.mean(widths), np.mean(heights)
        axes[i].scatter(avg_w, avg_h, color='red', s=100, marker='x', linewidth=3)
        axes[i].text(avg_w, avg_h, f'Avg: {avg_w:.0f}x{avg_h:.0f}', 
                    ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()


### Data Quality Assessment

Let's check for any potential issues with the dataset including corrupted images and class balance.


In [ ]:
# Check for corrupted images
corrupted_images = []
total_checked = 0

print("CHECKING FOR CORRUPTED IMAGES...")
print("="*40)

for class_name in classes:
    class_path = data_path / class_name
    if class_path.exists():
        images = list(class_path.glob('*.png'))
        
        for img_path in images:
            total_checked += 1
            try:
                img = Image.open(img_path)
                img.verify()  # Verify the image
            except Exception as e:
                corrupted_images.append((str(img_path), str(e)))

print(f"Total images checked: {total_checked}")
print(f"Corrupted images: {len(corrupted_images)}")

if corrupted_images:
    print("\nCorrupted images found:")
    for img_path, error in corrupted_images:
        print(f"  {img_path}: {error}")
else:
    print("\n✅ No corrupted images found!")

# Check class balance
print("\n" + "="*40)
print("CLASS BALANCE ANALYSIS")
print("="*40)

if dataset_stats:
    min_count = min(dataset_stats.values())
    max_count = max(dataset_stats.values())
    balance_ratio = max_count / min_count
    
    print(f"Minimum class count: {min_count}")
    print(f"Maximum class count: {max_count}")
    print(f"Balance ratio: {balance_ratio:.2f}")
    
    if balance_ratio <= 1.1:
        print("✅ Dataset is well balanced")
    elif balance_ratio <= 1.5:
        print("⚠️ Dataset has minor imbalance")
    else:
        print("❌ Dataset has significant imbalance")
        
    # Calculate imbalance percentages
    for class_name, count in dataset_stats.items():
        imbalance = ((count - min_count) / min_count) * 100
        print(f"{class_name}: {imbalance:+.1f}% from minimum")

print("\n" + "="*40)
print("DATA QUALITY SUMMARY")
print("="*40)
print(f"✅ Total images: {total_images:,}")
print(f"✅ Corrupted images: {len(corrupted_images)}")
print(f"✅ Classes: {len(classes)}")
print(f"✅ Image format: PNG")
print(f"✅ Color channels: RGB (3 channels)")
print("="*40)


### Summary and Next Steps

Based on the data exploration, we can summarize our findings and plan the next steps for the project.

**Key Findings:**
1. **Dataset Structure**: The dataset contains images organized in three folders (rock, paper, scissors)
2. **Class Distribution**: [To be filled after running the analysis]
3. **Image Characteristics**: [To be filled after running the analysis]
4. **Data Quality**: [To be filled after running the analysis]

**Next Steps:**
- Data preprocessing and normalization
- Train/validation/test split (70/20/10)
- Data augmentation strategies
- Model architecture design and implementation
- Hyperparameter tuning
- Model evaluation and analysis

**Project Requirements Addressed:**
✅ **Data Exploration**: Thorough exploration of the dataset with visualizations
✅ **Dataset Summary**: Comprehensive analysis of dataset characteristics
✅ **Data Quality Assessment**: Checking for corrupted images and class balance
